<a href="https://colab.research.google.com/github/lhb0803/helmet-project/blob/main/colab/construction_image_detection3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 10041, done.
remote: Total 10041 (delta 0), reused 0 (delta 0), pack-reused 10041
Receiving objects: 100% (10041/10041), 10.39 MiB | 29.48 MiB/s, done.
Resolving deltas: 100% (6961/6961), done.
/content/yolov5
     |████████████████████████████████| 596 kB 9.0 MB/s 


In [ ]:
import torch
from IPython.display import Image, clear_output

clear_output()
print(f"Setup complete. Using torch {torch.__version__} \
    ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.0+cu111     (Tesla P100-PCIE-16GB)


# Get Ready Data

* ACID : basic 
* Kaggle : hard-helmet data

## Download ACID Data

In [ ]:
import os
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile
from tqdm import tqdm

from google.colab import drive

import numpy as np
import pandas as pd

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%ls

CONTRIBUTING.md  Dockerfile  LICENSE    requirements.txt*  tutorial.ipynb
data/            export.py   models/    setup.cfg          utils/
detect.py        hubconf.py  README.md  train.py           val.py


In [ ]:
%cp -R ../drive/MyDrive/Helmet-Project/ACID.zip data/ACID.zip

In [ ]:
!unzip data/ACID.zip -d data/

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: data/ACID/ACID_Images/000453.jpg  
  inflating: data/ACID/ACID_Images/001902.jpg  
  inflating: data/ACID/ACID_Images/001803.jpg  
  inflating: data/ACID/ACID_Images/000442.jpg  
  inflating: data/ACID/ACID_Images/001337.jpg  
  inflating: data/ACID/ACID_Images/000890.jpg  
  inflating: data/ACID/ACID_Images/001693.jpg  
  inflating: data/ACID/ACID_Images/000120.jpg  
  inflating: data/ACID/ACID_Images/000047.jpg  
  inflating: data/ACID/ACID_Images/002565.jpg  
  inflating: data/ACID/ACID_Images/001513.jpg  
  inflating: data/ACID/ACID_Images/002264.jpg  
  inflating: data/ACID/ACID_Images/000283.jpg  
  inflating: data/ACID/ACID_Images/000956.jpg  
  inflating: data/ACID/ACID_Images/001651.jpg  
  inflating: data/ACID/ACID_Images/002682.jpg  
  inflating: data/ACID/ACID_Images/002096.jpg  
  inflating: data/ACID/ACID_Images/001852.jpg  
  inflating: data/ACID/ACID_Images/000041.jpg  
  inflating: data/ACID/ACID_Images/000769.jpg  
  in

## Download Kaggle Hard-Helemet data

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hblee0803","key":"06058e152527d4170c3773a1c4079312"}'}

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 4.0 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=4c07f15c425ade4f5def5c74cf0fe70c0a5d08938d8e8117c0dc360c1581c614
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets files -v andrewmvd/hard-hat-detection
!kaggle datasets download -d andrewmvd/hard-hat-detection -p ../datasets

name,size,creationDate
100% 1.22G/1.22G [00:11<00:00, 118MB/s]
100% 1.22G/1.22G [00:11<00:00, 115MB/s]


In [ ]:
!unzip -q ../datasets/hard-hat-detection.zip -d data/hard-hat-detection && rm ../datasets/hard-hat-detection.zip

## Preprocess data for YOLOv5 format

In [ ]:
classes = ['person', 'helmet', 'head', 'excavator', 'dump_truck', 'concrete_mixer_truck']

In [ ]:
ACID_IMG_DIR = 'data/ACID/ACID_Images/'
ACID_ANNOT_DIR = 'data/ACID/ACID_Annotations/'

HARD_HELMET_IMG_DIR = 'data/hard-hat-detection/images/'
HARD_HELMET_ANNOT_DIR = 'data/hard-hat-detection/annotations/'

IMG_DIR = 'data/construction/images/'
LABEL_DIR = 'data/construction/labels/'

def convert_annot(size , box):
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh

    return [x, y, w, h]

def save_txt_file(img_jpg_file_name, size, img_box):
    save_file_name = LABEL_DIR +  img_jpg_file_name + '.txt'
    print(save_file_name)
    with open(save_file_name ,'a+') as file_path:
        for box in img_box:

            cls_num = classes.index(box[0])

            new_box = convert_annot(size, box[1:])

            file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

        file_path.flush()
        file_path.close()

def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
   
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        
        img_box.append([cls_name, x1, y1, x2, y2])
  
    save_txt_file(img_xml_file, [img_w, img_h], img_box)

In [ ]:
!mkdir data/construction
!mkdir data/construction/labels
!mkdir data/construction/images

In [ ]:
# convert annotation to label
# ACID data

files = os.listdir(ACID_ANNOT_DIR)
for file in files:
    print("file name: ", file)
    file_xml = file.split(".")
    print(file_xml[0])
    get_xml_data(ACID_ANNOT_DIR, file_xml[0])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
001023
data/construction/labels/001023.txt
file name:  002210.xml
002210
data/construction/labels/002210.txt
file name:  000054.xml
000054
data/construction/labels/000054.txt
file name:  001204.xml
001204
data/construction/labels/001204.txt
file name:  000804.xml
000804
data/construction/labels/000804.txt
file name:  001709.xml
001709
data/construction/labels/001709.txt
file name:  000159.xml
000159
data/construction/labels/000159.txt
file name:  000477.xml
000477
data/construction/labels/000477.txt
file name:  001184.xml
001184
data/construction/labels/001184.txt
file name:  002589.xml
002589
data/construction/labels/002589.txt
file name:  001702.xml
001702
data/construction/labels/001702.txt
file name:  001218.xml
001218
data/construction/labels/001218.txt
file name:  002480.xml
002480
data/construction/labels/002480.txt
file name:  001623.xml
001623
data/construction/labels/001623.txt
file name:  000125.xml
000125
data/construction/labels/000125.t

In [ ]:
# convert annotation to label
# Hard-Helmet data

files = os.listdir(HARD_HELMET_ANNOT_DIR)
for file in files:
    print("file name: ", file)
    file_xml = file.split(".")
    print(file_xml[0])
    get_xml_data(HARD_HELMET_ANNOT_DIR, file_xml[0])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
hard_hat_workers3410
data/construction/labels/hard_hat_workers3410.txt
file name:  hard_hat_workers4315.xml
hard_hat_workers4315
data/construction/labels/hard_hat_workers4315.txt
file name:  hard_hat_workers745.xml
hard_hat_workers745
data/construction/labels/hard_hat_workers745.txt
file name:  hard_hat_workers873.xml
hard_hat_workers873
data/construction/labels/hard_hat_workers873.txt
file name:  hard_hat_workers1093.xml
hard_hat_workers1093
data/construction/labels/hard_hat_workers1093.txt
file name:  hard_hat_workers3458.xml
hard_hat_workers3458
data/construction/labels/hard_hat_workers3458.txt
file name:  hard_hat_workers2051.xml
hard_hat_workers2051
data/construction/labels/hard_hat_workers2051.txt
file name:  hard_hat_workers4231.xml
hard_hat_workers4231
data/construction/labels/hard_hat_workers4231.txt
file name:  hard_hat_workers1801.xml
hard_hat_workers1801
data/construction/labels/hard_hat_workers1801.txt
file name:  hard_hat_workers514.xml

## Train - Valid Split

In [ ]:
from sklearn.model_selection import train_test_split

# ACID
ACID_image_list = os.listdir(ACID_IMG_DIR)
ACID_train_list, ACID_valid_list = train_test_split(ACID_image_list, test_size=0.2, random_state=76)

# Hard-Helmet
HH_image_list = os.listdir(HARD_HELMET_IMG_DIR)
HH_train_list, HH_valid_list = train_test_split(HH_image_list, test_size=0.2, random_state=76)

print('total :',len(ACID_image_list) + len(HH_image_list))
print('train :',len(ACID_train_list) + len(HH_train_list))
print('val   :',len(ACID_valid_list) + len(HH_valid_list))

total : 7850
train : 6280
val   : 1570


In [ ]:
def copy_data(file_list, img_labels_root, imgs_source, mode):

    root_file = Path(IMG_DIR + mode)
    if not root_file.exists():
        print(f"Path {root_file} does not exist")
        os.makedirs(root_file)

    root_file = Path(LABEL_DIR + mode)
    if not root_file.exists():
        print(f"Path {root_file} does not exist")
        os.makedirs(root_file)

    for file in tqdm(file_list):
        img_name, img_format = file.split('.')
        img_src_file = imgs_source  + img_name + f".{img_format}"
        label_src_file = img_labels_root + img_name + '.txt'

        # Copy image
        img_dict_file = IMG_DIR + mode + '/' + img_name + f".{img_format}"
        copyfile(img_src_file, img_dict_file)
        # Delete image
        os.remove(img_src_file)

        # Copy label
        label_dict_file = LABEL_DIR + mode + '/' + img_name + '.txt'
        copyfile(label_src_file, label_dict_file)
        # Delete label
        os.remove(label_src_file)


In [ ]:
# ACID
copy_data(ACID_train_list, LABEL_DIR, ACID_IMG_DIR, "train")
copy_data(ACID_valid_list, LABEL_DIR, ACID_IMG_DIR, "val")

# Hard-Helmet
copy_data(HH_train_list, LABEL_DIR, HARD_HELMET_IMG_DIR, "train")
copy_data(HH_valid_list, LABEL_DIR, HARD_HELMET_IMG_DIR, "val")

Path data/construction/images/train does not exist
Path data/construction/labels/train does not exist


100%|██████████| 2280/2280 [00:01<00:00, 1709.44it/s]


Path data/construction/images/val does not exist
Path data/construction/labels/val does not exist


100%|██████████| 1000/1000 [00:01<00:00, 703.80it/s]


## Create YAML file for dataset configuration

In [ ]:
import yaml

data_config_dict = {
    'train': IMG_DIR + 'train',
    'val': IMG_DIR + 'val',
    'nc': '6',
    'names': classes
}

with open('data/construction.yaml', 'w+') as f:
    documents = yaml.dump(data_config_dict, f)

# Train

## Set visualization setting with W&B tool

In [ ]:
# Weights & Biases  (optional)
%pip install -q wandb
import wandb

     |████████████████████████████████| 1.7 MB 6.3 kB/s 
     |████████████████████████████████| 97 kB 8.5 MB/s 
     |████████████████████████████████| 140 kB 62.7 MB/s 
     |████████████████████████████████| 180 kB 94.4 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Train with construction equipments

In [ ]:
# YOLO v5s
!python train.py --hyp hyp.finetune.yaml --img 416 --batch 32 --epochs 32 --data data/construction.yaml --cfg models/yolov5s.yaml --weights yolov5s.pt --name yolo5s_construction

wandb: Currently logged in as: hyob (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=data/construction.yaml, hyp=hyp.finetune.yaml, epochs=32, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=yolo5s_construction, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-110-gfcd180d torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.0032, lrf=0.12, momentum=0.843, weight_decay=0.00036, warmup_epochs=2.0, warmup_momentum=0.5, warmup_bias_lr=0.05, box=0.0296, cls=0.243, cls_pw=0.6

In [ ]:
# YOLO v5m
!python train.py --hyp hyp.finetune.yaml --img 416 --batch 32 --epochs 32 --data data/construction.yaml --cfg models/yolov5m.yaml --weights yolov5m.pt --name yolo5m_construction

wandb: Currently logged in as: hyob (use `wandb login --relogin` to force relogin)
train: weights=yolov5m.pt, cfg=models/yolov5m.yaml, data=data/construction.yaml, hyp=hyp.finetune.yaml, epochs=32, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=yolo5m_construction, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-110-gfcd180d torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.0032, lrf=0.12, momentum=0.843, weight_decay=0.00036, warmup_epochs=2.0, warmup_momentum=0.5, warmup_bias_lr=0.05, box=0.0296, cls=0.243, cls_pw=0.6

## Detect
`python detect.py --weights runs/train/yolo5s_construction/weights/best.pt --source 'https://youtu.be/MHaZXSneOGQ'`


## Download weights

In [ ]:
from google.colab import files

In [ ]:
!zip -r yolo5s_construction.zip runs/train/yolo5s_construction/
files.download('yolo5s_construction.zip')

  adding: runs/train/yolo5s_construction/ (stored 0%)
  adding: runs/train/yolo5s_construction/train_batch2.jpg (deflated 5%)
  adding: runs/train/yolo5s_construction/events.out.tfevents.1638092458.3545b832a1eb.474.0 (deflated 24%)
  adding: runs/train/yolo5s_construction/hyp.yaml (deflated 42%)
  adding: runs/train/yolo5s_construction/val_batch2_labels.jpg (deflated 8%)
  adding: runs/train/yolo5s_construction/opt.yaml (deflated 47%)
  adding: runs/train/yolo5s_construction/labels.jpg (deflated 16%)
  adding: runs/train/yolo5s_construction/val_batch0_pred.jpg (deflated 10%)
  adding: runs/train/yolo5s_construction/weights/ (stored 0%)
  adding: runs/train/yolo5s_construction/weights/last.pt (deflated 8%)
  adding: runs/train/yolo5s_construction/weights/best.pt (deflated 8%)
  adding: runs/train/yolo5s_construction/val_batch2_pred.jpg (deflated 7%)
  adding: runs/train/yolo5s_construction/val_batch1_pred.jpg (deflated 7%)
  adding: runs/train/yolo5s_construction/labels_correlogram.jpg 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r yolo5m_construction.zip runs/train/yolo5m_construction/
files.download('yolo5m_construction.zip')

  adding: runs/train/yolo5m_construction/ (stored 0%)
  adding: runs/train/yolo5m_construction/train_batch2.jpg (deflated 3%)
  adding: runs/train/yolo5m_construction/hyp.yaml (deflated 42%)
  adding: runs/train/yolo5m_construction/val_batch2_labels.jpg (deflated 8%)
  adding: runs/train/yolo5m_construction/opt.yaml (deflated 47%)
  adding: runs/train/yolo5m_construction/labels.jpg (deflated 16%)
  adding: runs/train/yolo5m_construction/val_batch0_pred.jpg (deflated 10%)
  adding: runs/train/yolo5m_construction/weights/ (stored 0%)
  adding: runs/train/yolo5m_construction/weights/last.pt (deflated 8%)
  adding: runs/train/yolo5m_construction/weights/best.pt (deflated 8%)
  adding: runs/train/yolo5m_construction/val_batch2_pred.jpg (deflated 7%)
  adding: runs/train/yolo5m_construction/val_batch1_pred.jpg (deflated 7%)
  adding: runs/train/yolo5m_construction/labels_correlogram.jpg (deflated 14%)
  adding: runs/train/yolo5m_construction/train_batch0.jpg (deflated 3%)
  adding: runs/trai

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>